# GENERATE DATASET

In [ ]:
#1. Generate dataset
#2. Train the classifier and save it
#3. Detect the face and named it if it is already stored in our dataset

In [1]:
import cv2
import os

In [4]:
import cv2

def generate_dataset():
    # Load Haar cascades
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    profile_classifier = cv2.CascadeClassifier("haarcascade_profileface.xml")

    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # 1. Detect frontal faces
        faces = face_classifier.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=4,
            minSize=(50, 50)
        )
        if len(faces) > 0:
            # Detected frontal
            return faces[0], False

        # 2. Detect right-profile faces
        faces = profile_classifier.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=4,
            minSize=(50, 50)
        )
        if len(faces) > 0:
            return faces[0], False

        # 3. Detect left-profile faces by flipping image
        flipped_gray = cv2.flip(gray, 1)
        faces = profile_classifier.detectMultiScale(
            flipped_gray,
            scaleFactor=1.1,
            minNeighbors=4,
            minSize=(50, 50)
        )
        if len(faces) > 0:
            # Get the first detected
            x, y, w, h = faces[0]
            # Convert x back to original image coordinates
            x = gray.shape[1] - x - w
            return (x, y, w, h), True

        # No face detected
        return None, False

    cap = cv2.VideoCapture(0)
    user_id = 2  # Change as needed
    img_id = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        result, flipped = face_cropped(frame)
        if result is not None:
            x, y, w, h = result
            crop = frame[y:y+h, x:x+w]
            # Resize but keep the RGB colors
            face_resized = cv2.resize(crop, (300, 300))
            
            img_id += 1
            filename = f"data/kimly.{user_id}.{img_id}.jpg"
            cv2.imwrite(filename, face_resized)  # Save the color image directly

            # Annotate detection type
            label = f"{img_id} - {'L' if flipped else 'F'}"
            cv2.putText(
                face_resized, label, (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2
            )
            cv2.imshow("Cropped face", face_resized)

        key = cv2.waitKey(1)
        if key == 27 or img_id >= 150:
            break

    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples completed.")

if __name__ == '__main__':
    generate_dataset()


In [21]:
%pip install pillow
%pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import os
import cv2
from PIL import Image #pip install pillow
import numpy as np    # pip install numpy
 
def train_classifier(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
     
    faces = []
    ids = []
     
    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
         
        faces.append(imageNp)
        ids.append(id)
         
    ids = np.array(ids)
     
    # Train and save classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")
train_classifier("data")

PermissionError: [Errno 13] Permission denied: 'data\\an'

In [ ]:
import cv2
import numpy as np
from PIL import Image
import os
 
def draw_boundary(img, classifier, profile_classifier, scaleFactor, minNeighbors, color, text, clf):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)

    if len(features) == 0:
        features = profile_classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
        if len(features) == 0:
            flipped_gray = cv2.flip(gray_img, 1)
            features = profile_classifier.detectMultiScale(flipped_gray, scaleFactor, minNeighbors)
     
    for (x,y,w,h) in features:
        cv2.rectangle(img, (x,y), (x+w,y+h), color, 2 )
         
        id, pred = clf.predict(gray_img[y:y+h,x:x+w])
        confidence = int(100*(1-pred/300))
         
        if confidence>70:
            if id==1:
                cv2.putText(img, "Xuan Phuc", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            if id==2:
                cv2.putText(img, "Cristiano Ronaldo", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            cv2.putText(img, "UNKNOWN", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 1, cv2.LINE_AA)
     
    return img
 
# loading classifier
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
profileCascade = cv2.CascadeClassifier("haarcascade_profileface.xml")

clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")
 
video_capture = cv2.VideoCapture(0)
 
while True:
    ret, img = video_capture.read()
    img = draw_boundary(img, faceCascade, profileCascade, 1.4, 6, (0, 255, 0), "Face", clf)
    cv2.imshow("face Detection", img)
     
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()

error: OpenCV(4.11.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


: 